<a href="https://colab.research.google.com/github/PetersServers/Assignments/blob/main/Digital_orga_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
data_table.enable_dataframe_formatter()

drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/investmentdata_4regression(1).csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filtered_data = data[data['investdum'].astype(bool) == False]

In [52]:
unique_combinations = data.groupby(['homeC', 'targetC']).size().reset_index(name='count')
len(unique_combinations)

456

In [54]:
data_country_year = pd.DataFrame()  

for country in data['targetC'].unique():
    country_data = data[data['targetC'] == country]
    
    for year in data["year"].unique():
        year_data = country_data[country_data['year'] == year]
        gdpt_growth = year_data['GDPgrowth_p.a.prc'].values[0]
        gdp_usd = year_data['GDP.USD'].values[0]
      
        polconi = year_data['polconiii'].values[0]

         
        data_country_year = data_country_year.append({'targetC': country, 'GDPgrowth_p.a.prc': gdpt_growth, 'GDP.USD': gdp_usd, 'year': year, 'polconiii': polconi}, ignore_index=True)

<ipython-input-54-179bb87aab52>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_country_year = data_country_year.append({'targetC': country, 'GDPgrowth_p.a.prc': gdpt_growth, 'GDP.USD': gdp_usd, 'year': year, 'polconiii': polconi}, ignore_index=True)
<ipython-input-54-179bb87aab52>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_country_year = data_country_year.append({'targetC': country, 'GDPgrowth_p.a.prc': gdpt_growth, 'GDP.USD': gdp_usd, 'year': year, 'polconiii': polconi}, ignore_index=True)
<ipython-input-54-179bb87aab52>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_country_year = data_country_year.append({'targetC': country, 'GDPgrowth_p.a.prc': gdpt_growth, 'GDP.USD': gdp_usd, 'year': year, '

In [57]:
data_countries_distance = data[['homeC', 'targetC', 'distance']].copy()
data_countries_distance = data_countries_distance.drop_duplicates()

Create a df of all the unique companies data for each unique year 

In [58]:
data_company_sales_per_year = pd.DataFrame()

for comp_id in data['comp_id'].unique():
    comp_data = data[data['comp_id'] == comp_id]
    
    for year in comp_data['year'].unique():
        year_data = comp_data[comp_data['year'] == year]
        prev_year = comp_data[comp_data['year'] == year - 1]
        sales = year_data['sales'].values[0]
        profit = year_data['profit'].values[0]
        assets = year_data['assets'].values[0]
        home_country = year_data['homeC'].values[0]
         
        data_company_sales_per_year = data_company_sales_per_year.append({
            'comp_id': comp_id,
            'year': year,
            'sales': sales,
            'profit': profit, 
            'assets': assets, 
            'homeC': home_country
        }, ignore_index=True)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
<ipython-input-58-175d39e02476>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_company_sales_per_year = data_company_sales_per_year.append({
<ipython-input-58-175d39e02476>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_company_sales_per_year = data_company_sales_per_year.append({
<ipython-input-58-175d39e02476>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_company_sales_per_year = data_company_sales_per_year.append({
<ipython-input-58-175d39e02476>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_company_sales_per_year = data_company_sales

Merge yearly company data with the distances of home country to target country

In [ ]:
countries_distance_company_merged = pd.merge(data_countries_distance, data_company_sales_per_year, on='homeC')

merge the data of the countries relating to the target countries data with the data of the companies 

In [ ]:
merged_df = pd.merge(countries_distance_company_merged, data_country_year, on=['targetC', 'year'])

declare the investment dummies of all the combinations that we found. 

In [ ]:
merged_df["previous_year"] = merged_df["year"] -1
merged_df["investdum"] = False 

Filter out the investments that have acutally been made (filter out the combinations that are True in the original data)

In [ ]:
conditions = (
    merged_df['targetC'].isin(data['targetC']) &
    merged_df['year'].isin(data['year']) &
    merged_df['comp_id'].isin(data['comp_id']) &
    (merged_df['investdum'] == False) &
    (data['investdum'].astype(bool) == True)
)

filtered_merged_df = merged_df.drop(merged_df[conditions].index)


<ipython-input-46-f827de24627e>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_merged_df = merged_df.drop(merged_df[conditions].index)


This dataset now contains only investments that haven't been made

In [ ]:
filtered_merged_df

,homeC,targetC,distance,comp_id,year,previous_year,sales,profit,assets,GDPgrowth_p.a.prc,GDP.USD,polconiii,investment_dum,investmentdum,investdum
0,GBR,AUS,15310.944960,GB07289402,2008,2007,8.897338e+06,233847.431700,5.743778e+06,3.84423,8.530996e+11,0.489033,False,False,False
1,GBR,AUS,15310.944960,GB06497379,2008,2007,9.228591e+06,298506.861700,2.271457e+07,3.84423,8.530996e+11,0.489033,False,False,False
2,GBR,AUS,15310.944960,GB09198021,2008,2007,2.262037e+06,198721.427000,6.157956e+06,3.84423,8.530996e+11,0.489033,False,False,False
3,GBR,AUS,15310.944960,GB06388542,2008,2007,9.228591e+06,342999.137900,1.010155e+07,3.84423,8.530996e+11,0.489033,False,False,False
4,GBR,AUS,15310.944960,GB05817296,2008,2007,7.312673e+01,-11369.175800,8.764187e+01,3.84423,8.530996e+11,0.489033,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329540,AUS,GBR,15310.944960,AU108787720,2009,2008,1.251384e+03,-3559.705353,1.734707e+04,-0.27878,2.921432e+12,0.396922,False,False,False
329541,AUS,GBR,15310.944960,AU612180355,2009,2008,8.449728e+05,241324.535400,9.273331e+06,-0.27878,2.921432e+12,0.396922,False,False,False
329542,KOR,GBR,8811.401318,KR1101110924765,2009,2008,3.494608e+07,860805.263800,3.103427e+08,-0.27878,2.921432e+12,0.396922,False,False,False
329543,AUT,GBR,1307.720732,AT9110283855,2009,2008,4.030000e+00,-4716.710000,9.321000e+01,-0.27878,2.921432e+12,0.396922,False,False,False
